## Домашнее задание к лекции «Продвинутый pandas 2»
### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [83]:
import pandas as pd
from datetime import datetime
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings_by_user = ratings[['userId','rating']]
ratings_by_user.head(3)


,userId,rating
0,1,2.5
1,1,3.0
2,1,3.0


In [122]:
users_count = ratings_by_user.groupby('userId').count()
users_count.head()
users_more_100 = users_count[users_count.rating > 100] # Формируем датафрейм с пользователями больше 100 оценок
users_more_100.head()

,rating
userId,
4,204
8,116
15,1700
17,363
19,423


In [123]:
timestamp_min = ratings.groupby('userId')['timestamp'].min() # Датафрейм с минимальным timestamp
timestamp_min.head(3)

userId
1    1260759108
2     835355395
3    1298861589
Name: timestamp, dtype: int64

In [124]:
timestamp_max = ratings.groupby('userId')['timestamp'].max() # Датафрейм с максимальным timestamp
timestamp_max.head(3)

userId
1    1260759205
2     835356246
3    1298932787
Name: timestamp, dtype: int64

In [127]:
users_ = users_more_100.join(timestamp_min) # объединяем таблицы в одну
users_.rename(columns={'timestamp': 'timestamp_min'},inplace=True)
users_ = users_.join(timestamp_max)
users_.rename(columns={'timestamp': 'timestamp_max'},inplace=True)
users_['diff_timestamp'] = users_['timestamp_max'] - users_['timestamp_min'] # считаем время жизни
users_.head()

,rating,timestamp_min,timestamp_max,diff_timestamp
userId,,,,
4,204,949778714,949982274,203560
8,116,1154389340,1154474527,85187
15,1700,997937239,1469330735,471393496
17,363,1127468587,1127476640,8053
19,423,855190091,855195373,5282


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [128]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [129]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [130]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [164]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [194]:
def add_revenue(base_table, *args):
    """ Функция склеивает базовую таблицу с аргументами без потерь"""
    for i in args:
        base_table = base_table.merge(i, how = 'outer')
    return base_table
client_base_ = add_revenue(client_base, rzd, auto, air)
client_base_.drop(['address'], axis = 'columns', inplace = True)
client_base_ # первая таблица

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [197]:
client_base_2 = add_revenue(client_base, rzd, auto, air)
client_base_2 = client_base_2.groupby('address')[['address', 'rzd_revenue', 'auto_revenue', 'air_revenue']]
client_base_2.head(10) # вторая таблица

,address,rzd_revenue,auto_revenue,air_revenue
0,Комсомольская 4,1093.0,NaN,NaN
1,Энтузиастов 8а,2810.0,NaN,NaN
2,Левобережная 1а,10283.0,57483.0,NaN
3,Мира 14,5774.0,83.0,NaN
4,ЗЖБИиДК 1,981.0,912.0,81.0
5,Строителей 18,NaN,4834.0,4.0
6,Панфиловская 33,NaN,98.0,13.0
7,Мастеркова 4,NaN,NaN,173.0


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

### Ответ:
Если определить небольшую дельту по координатам (+- 500 метров), то отсортировав по колличеству попавших координат в топ можно отпределить примерный адрес проживания или работы. 
Если бы была привязка ко времени, было бы проще.
Достаточно расчитать две наиболее часто встречающиеся точки, и определив район города(или улицу), добавить в таблицу. 
Тогда можно будет поискать зависимость частоты визитов\покупок от расстояния до этих точек.

Если координаты значительно удалены друг от друга, то человек живёт далеко от работы (или у него любовница живёт на другом конце города). 

Дополнительные признаки которые можно получить из координат:
1. Как часто в день покупатель воспользовался приложением?
2. Есть ли связь между колличеством координат(тоесть использованием приложения) и визитами\покупками?
3. С каких районов\улиц приходит больше всего покупателей?
4. Если координат много, можно попробовать нарисовать трек передвижения покупателя.
5. Как часто покупатель бывает по координатам основных конкурентов?
6. Какова вероятность, что покупатель с id=129444 работает в Роспотребнадзоре по адресу ул. Космонавтов 14?
